# 🚀 18QUAD LoRA 学習テンプレート
このノートブックでは、LoRAを使って18QUADに特化した言語モデルの微調整を行います。

In [ ]:
# ✅ 必要なライブラリのインストール
!pip install -q transformers datasets peft accelerate bitsandbytes

In [ ]:
# ✅ 学習用データの読み込み（JSONL形式）
from datasets import load_dataset
dataset = load_dataset('json', data_files='18quad_lora_train.jsonl', split='train')
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
# ✅ モデルとLoRA設定の読み込み
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

model_name = 'meta-llama/Llama-2-7b-hf'  # or any other compatible model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

In [ ]:
# ✅ 学習準備とTrainer設定
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./lora-18quad",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()